In [ ]:
# Water-Net: flood extraction network combining Transformer and CNN from SAR
# Author: Teng Zhao, Xiaoping Du and Xiangtao Fan.
# Version: 1.0
# Date:27/11/2022
#导入相关库
import datetime
import time
import shapefile
import pygeoif
from snappy import ProductIO
from snappy import HashMap
from snappy import GPF
from snappy import File
from snappy import ProgressMonitor
import snappy
import sys
import os

In [ ]:
#定义预处理操作——使用SNAP下的snappy库处理：
###读取哨兵文件zip压缩格式等
def read_s1_zip_file(file_name):
    print('\tReading Sentinel-1 zip file...')
    # 执行读入操作
    output = ProductIO.readProduct(file_name)
    return output
###进行轨道校正
def do_apply_orbit_file(product):
    print('\tapplyOrbit...')
    parameters = HashMap()
    parameters.put('orbitType', 'Sentinel Precise (Auto Download)')
    parameters.put('polyDegree', '3')
    parameters.put('continueOnFail', 'false')
    # 执行轨道校正操作
    output = GPF.createProduct('Apply-Orbit-File', parameters, product)
    return output

###进行噪声去除操作：
def do_thermal_noise_removal(product):
    # 热噪声去除操作 - snappy
    parameters = HashMap()
    parameters.put('removeThermalNoise', True)
    # 执行热噪声去除操作
    output = GPF.createProduct('ThermalNoiseRemoval', parameters, product)
    return output

###进行辐射定标：
def do_calibration(product):
    print('\tCalibration...')
    parameters = HashMap()
    # 以标准后向散射系数sigma0的方式定标
    parameters.put('outputSigmaBand', True)
    parameters.put('sourceBands', 'Intensity_VH,Intensity_VV')
    parameters.put('selectedPolarisations', 'VH,VV')
    # 没有分贝化
    parameters.put('outputImageScaleInDb', False)
    output = GPF.createProduct("Calibration", parameters, product)
    return output

###进行滤波处理：参数有滤波器设置
def do_speckle_filtering(product):
    print('\tSpeckle filtering...')
    parameters = HashMap()
    # 使用Refine Lee滤波器。默认是Lee Sigma
    parameters.put('filter','Refined Lee')
    output = GPF.createProduct('Speckle-Filter', parameters, product)
    return output

###进行地形校正（地理编码）操作：
def do_terrain_correction(product, proj=""):
    print('\tTerrain correction...')
    parameters = HashMap()
    parameters.put('demName', 'SRTM 1Sec HGT')
    parameters.put('imgResamplingMethod', 'BILINEAR_INTERPOLATION')
   # parameters.put('mapProjection', proj)       # 如果你需要使用UTM/WGS84投影，可以注释该行代码,默认是WGS84地理坐标系
    parameters.put('saveProjectedLocalIncidenceAngle', False)
    parameters.put('saveSelectedproductBand', True)
    parameters.put('nodataValueAtSea', False)
    # 分辨率为10m
    #parameters.put('pixelSpacingInMeter', 10.0)
    output = GPF.createProduct('Terrain-Correction', parameters, product)
    return output

###进行分贝化处理：
def do_line_to_db(product):#将线性单位转为分贝值
    print('\tLining_to_db...')
    # 使用默认参数值，即选择所有通道
    parameters = HashMap()
    output = GPF.createProduct('LinearToFromdB',parameters,product)
    return output

# 使用wkt坐标范围裁剪:
def shpToWKT(shp_path):
    r=shapefile.Reader(shp_path,encoding="GBK")
    g=[]
    for s in r.shapes():
        g.append(pygeoif.geometry.as_shape(s))
    m=pygeoif.MultiPoint(g)
    return str(m.wkt).replace("MULTIPOINT","POLYGON(")+")"
def subset(product,shpPath):
    parameters=HashMap()
    wkt=shpToWKT(shpPath)
    #SubsetOp=snappy.jpy.get_type('org.esa.snap.core.gpf.common.SubsetOp')
    geometry=snappy.WKTReader().read(wkt)
    parameters.put('copyMetadata',True)
    parameters.put('geoRegion',geometry)
    return GPF.createProduct('Subset',parameters,product)

# 使用像素坐标裁剪
def do_subset(product, x, y, width, height):
    parameters = HashMap()
    # 复制元数据
    parameters.put('copyMetadata', True)
    # 设置裁剪区域参数
    parameters.put('region', '%s, %s, %s, %s' % (x, y, width, height))
    # 执行裁剪操作
    output = GPF.createProduct('Subset', parameters, product)
    return output

###进度条工具
def createProgressMonitor():
    PWPM= snappy.jpy.get_type('com.bc.ceres.core.PrintWriterProgressMonitor')
    JavaSystem=snappy.jpy.get_type('java.lang.System')
    monitor=PWPM(JavaSystem.out)
    return monitor

###录入结果文件：
def write_to_file(product, file_path, format='BEAM-MAP'):
    print("Writing...")
    # 不支持更新数据
    incremental = False
    pm=createProgressMonitor()
    # 写入操作 - snappy
    # 以以format格式写入
    GPF.writeProduct(product, File(file_path), format, incremental,pm)

In [ ]:
#影像预处理测试：
product=read_s1_zip_file("数据源位置")
product_subset=subset(product,"矢量文件位置")##裁剪  减少数据处理量
del product
product_Subset_Orb = do_apply_orbit_file(product_subset)   #轨道校准
del product_subset
product_subset_Orb_TNR=do_thermal_noise_removal(product_Subset_Orb)  #热噪声去除
del product_Subset_Orb
product_subset_Orb_TNR_Cal=do_calibration(product_subset_Orb_TNR)  #辐射定标
del product_subset_Orb_TNR
product_subset_Orb_TNR_Cal_TC=do_terrain_correction(product_subset_Orb_TNR_Cal)   #地形校正
del product_subset_Orb_TNR_Cal
product_subset_Orb_TNR_Cal_TC_dB = do_line_to_db(product_subset_Orb_TNR_Cal_TC) #分贝化处理
del product_subset_Orb_TNR_Cal_TC
product_subset_TNR_Cal_TC_spk=do_speckle_filtering(product_subset_Orb_TNR_Cal_TC_dB)#滤波处理
del product_subset_Orb_TNR_Cal_TC_dB
write_to_file(product_subset_TNR_Cal_TC_spk,"保存位置","GeoTIFF")
del product_subset_TNR_Cal_TC_spk
